<a href="https://colab.research.google.com/github/anshupandey/Working_with_Large_Language_models/blob/main/WWL_C6_transformer_based_classification_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers --quiet

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

# load dataset
url = "https://raw.githubusercontent.com/anshupandey/Working_with_Large_Language_models/main/sentimentdata.csv"
df = pd.read_csv(url)

x = df['document']
y = df['label']

# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')




# Convert texts to embeddings
X = model.encode(x)
y = np.array(y)


# Create and train the logistic regression model
clf = LogisticRegression()
clf.fit(X, y)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LogisticRegression()

In [8]:
ip = "Noodles are good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype=object)

In [9]:
ip = "Noodles are not good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [10]:
ip = "Noodles are bad"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [11]:
ip = "Noodles are not good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [12]:
ip = "Noodles are great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype=object)

In [13]:
ip = "Noodles are not great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)